In [1]:
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import warnings
warnings.filterwarnings("ignore")
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pysptk
from scipy.io import wavfile
import warnings
import speech_recognition as sr9
from nltk.tokenize import word_tokenize
# nltk.download('punkt')  # 처음한번 필요
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))
r = sr9.Recognizer()


In [3]:
def editDistance(r, h):
    d = np.zeros((len(r) + 1) * (len(h) + 1), dtype=np.uint8).reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        d[i][0] = i
    for j in range(len(h) + 1):
        d[0][j] = j
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitute = d[i - 1][j - 1] + 1
                insert = d[i][j - 1] + 1
                delete = d[i - 1][j] + 1
                d[i][j] = min(substitute, insert, delete)
    return d



def getStepList(r, h, d):
    x = len(r)
    y = len(h)
    list9 = []
    while True:
        if x == 0 and y == 0:
            break
        elif x >= 1 and y >= 1 and d[x][y] == d[x - 1][y - 1] and r[x - 1] == h[y - 1]:
            list9.append("e")
            x = x - 1
            y = y - 1
        elif y >= 1 and d[x][y] == d[x][y - 1] + 1:
            list9.append("i")
            x = x
            y = y - 1
        elif x >= 1 and y >= 1 and d[x][y] == d[x - 1][y - 1] + 1:
            list9.append("s")
            x = x - 1
            y = y - 1
        else:
            list9.append("d")
            x = x - 1
            y = y
    global kinds_of_error
    kinds_of_error = list9[::-1]
    return list9[::-1]


def alignedPrint(list9, r, h, result):
    Ted_words = []
    User_words = []
    print("Speecher words:", end=" ")
    for i in range(len(list9)):
        if list9[i] == "i":
            count = 0
            for j in range(i):
                if list9[j] == "d":
                    count += 1
            index = i - count
            # print(" "*(len(h[index])), end=" ")
            Ted_words.append(" " * (len(h[index])))
        elif list9[i] == "s":
            count1 = 0
            for j in range(i):
                if list9[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list9[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) < len(h[index2]):
                Ted_words.append(r[index1] + " " * (len(h[index2]) - len(r[index1])))
                # print(r[index1] + " " * (len(h[index2])-len(r[index1])), end=" ")
            else:
                Ted_words.append(r[index1])
                # print(r[index1], end=" "),
        else:
            count = 0
            for j in range(i):
                if list9[j] == "i":
                    count += 1
            index = i - count
            Ted_words.append(r[index])
            print(r[index], end=" "),
    print("\nShadowed words:", end=" ")
    for i in range(len(list9)):
        if list9[i] == "d":
            count = 0
            for j in range(i):
                if list9[j] == "i":
                    count += 1
            index = i - count
            User_words.append(" " * (len(r[index])))
            print(" " * (len(r[index])), end=" ")
        elif list9[i] == "s":
            count1 = 0
            for j in range(i):
                if list9[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list9[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                User_words.append(h[index2] + " " * (len(r[index1]) - len(h[index2])))
                print(h[index2] + " " * (len(r[index1])-len(h[index2])), end=" ")
            else:
                User_words.append(h[index2])
                print(h[index2], end=" ")
        else:
            count = 0
            for j in range(i):
                if list9[j] == "d":
                    count += 1
            index = i - count
            User_words.append(h[index])
            print(h[index], end=" ")
    print("\ntypes of wrong:", end=" ")
    for i in range(len(list9)):
        if list9[i] == "d":
            count = 0
            for j in range(i):
                if list9[j] == "i":
                    count += 1
            index = i - count
            print("D" + " " * (len(r[index])-1), end=" ")
        elif list9[i] == "i":
            count = 0
            for j in range(i):
                if list9[j] == "d":
                    count += 1
            index = i - count
            print("I" + " " * (len(h[index])-1), end=" ")
        elif list9[i] == "s":
            count1 = 0
            for j in range(i):
                if list9[j] == "i":
                    count1 += 1
            index1 = i - count1
            count2 = 0
            for j in range(i):
                if list9[j] == "d":
                    count2 += 1
            index2 = i - count2
            if len(r[index1]) > len(h[index2]):
                print("S" + " " * (len(r[index1]) - 1), end=" ")
            else:
                print("S" + " " * (len(h[index2]) - 1), end=" ")
        else:
            count = 0
            for j in range(i):
                if list9[j] == "i":
                    count += 1
            index = i - count
            print(" " * (len(r[index])), end=" ")
    print("\nCorrect Rate: " + result)
    print('!!',Ted_words,'!!')
    print('!!!!',User_words,'!!!!')
    return Ted_words, User_words


#(mine_token,ted_token)
def wer(r, h):
    d = editDistance(r, h)

    list_final = getStepList(r, h, d)

    result = float(d[len(r)][len(h)]) / max(len(h), len(r)) * 100

    result = round(100 - result, 2)
    result_str = str("%.2f" % result) + "%"
    Ted_words, User_words = alignedPrint(list_final, r, h, result_str)
    print('D: 놓친단어, S: 잘못말한 단어, I: 원본에 없는 단어')
    return result, Ted_words, User_words
    return {"result":result, "Ted_words":Ted_words, "User_words":User_words}

# 구글 stt와 문장 비교.
def eval_pronounciation(ted_audio_path, user_audio_path):
    r = sr9.Recognizer()
    audio = None
    ted_answer = None
    your_answer = None

    with sr9.AudioFile(ted_audio_path) as source:
        audio = r.record(source)
        ted_answer = r.recognize_google(audio)

    answer = r.recognize_google(audio)
    answer = answer.lower()
    with sr9.AudioFile(user_audio_path) as source:
        audio = r.record(source)
        your_answer = r.recognize_google(audio)
    mine = r.recognize_google(audio)
    mine = mine.lower()
    answer_token = word_tokenize(answer)
    mine_token = word_tokenize(mine)

    result, Ted_words, User_words = wer(mine_token, answer_token)
    global pronounciation_result
    if result > 80:
        pronounciation_result = 'Excellent'
    elif result > 60:
        pronounciation_result = 'Good'
    else:
        pronounciation_result = 'Bad'
    return ted_answer, your_answer, result, pronounciation_result, kinds_of_error, Ted_words, User_words


In [7]:
ted_audio_path='C:/Users/todtj/OneDrive/바탕 화면/비교/21_3.wav'
user_audio_path='C:/Users/todtj/OneDrive/바탕 화면/비교/jungeui_21_3.wav'

In [8]:
eval_pronounciation(ted_audio_path, user_audio_path)

Speecher words: but then i thinking about my family 
Shadowed words: but      i started thinking about my family essay 
types of wrong:     D      S                                I     
Correct Rate: 62.50%
!! ['but', 'then', 'i', 'start  ', 'thinking', 'about', 'my', 'family', '     '] !!
!!!! ['but', '    ', 'i', 'started', 'thinking', 'about', 'my', 'family', 'essay'] !!!!
D: 놓친단어, S: 잘못말한 단어, I: 원본에 없는 단어


('but I started thinking about my family essay',
 'but then I start thinking about my family',
 62.5,
 'Good',
 ['e', 'd', 'e', 's', 'e', 'e', 'e', 'e', 'i'],
 ['but', 'then', 'i', 'start  ', 'thinking', 'about', 'my', 'family', '     '],
 ['but', '    ', 'i', 'started', 'thinking', 'about', 'my', 'family', 'essay'])